## Experiments with linopy and pyomo


A post office requires a different number of full time employees on different days of the week: Each worker must work 5 consecutive days and then have 2 days off. The office wants to meet its minimum requirement for workers. Write an LP to minimize the number of workers to be hired.

Sets:
$$D = \text{set of week days}$$
$$W = \text{set of consecutive working days}$$
Variables:
$$x \in \mathbb{Z}^+$$
Objective:
minimize the total number of employees
$$min_x\text{ z} = \sum_{d \in D} x_d$$
subject to:
$$\sum_{w \in W} x_{(d+w)} \mod 7 \geq \epsilon _d \quad \forall d \in D$$
$$\epsilon_d:\text{ number of full time employees per day }$$  
with:
<table style="margin-left: auto;margin-right: auto;">
<caption>number of full time employees per day</caption>
  <tr>
    <td>Monday</td>
    <td>Tuesday</td>
    <td>Wednesday</td>
    <td>Thursday</td>
    <td>Friday</td>
    <td>Saturday</td>
    <td>Sunday</td>
  </tr>
  <tr>
    <td>17</td>
    <td>13</td>
    <td>15</td>
    <td>19</td>
    <td>14</td>
    <td>16</td>
    <td>11</td>
  </tr>
</table>


In [10]:
d = range(7)

def test(d: int):
    return [ (d + i)%7 for i in range(5)]

for day in d:
    print(f"{day}: {test(day)}")

0: [0, 1, 2, 3, 4]
1: [1, 2, 3, 4, 5]
2: [2, 3, 4, 5, 6]
3: [3, 4, 5, 6, 0]
4: [4, 5, 6, 0, 1]
5: [5, 6, 0, 1, 2]
6: [6, 0, 1, 2, 3]


## Pyomo Solution

In [19]:
import pyomo.environ as pyomo
model = pyomo.ConcreteModel()

# the days of the week, starting at monday
model.d = pyomo.Set(initialize = range(7))
# consecutive working days
model.wd = pyomo.Set(initialize = range(5))

# number of starting workers per day
model.x = pyomo.Var(model.d, within=pyomo.NonNegativeIntegers )

# number of full time employees per day
model.epsilon = pyomo.Param(
    model.d,
    initialize={
        0: 17,
        1: 13,
        2: 15,
        3: 19,
        4: 14,
        5: 16,
        6: 11,
    } 
)


model.obj = pyomo.Objective(expr = sum(model.x[d] for d in model.d), sense=pyomo.minimize)


def rule1(model, d):
    return sum(model.x[(d+ working_day)%7]  for working_day in model.wd)  >= model.epsilon[d]

model.eq1 = pyomo.Constraint(model.d, rule=rule1, doc="full time employee requirement")


optimizer = pyomo.SolverFactory('appsi_highs')
log = optimizer.solve(model)

log.write()

print(f"number of overall employees: {model.obj()}")

model.pprint()



Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d]
Copyright (c) 2023 HiGHS under MIT licence terms
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: 23.0
  Upper bound: 23.0
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 0
  Sense: 1
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Termination message: TerminationCondition.optimal
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of s

# Linopy Solution